In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import SeparableConv2D
from tensorflow.keras.layers import Add

# Convolution with Batch Norm & ReLU

In [2]:
class ConvBNReLU(Model):   
    def __init__(self, filters, kernel_size, inputs):
        super().__init__()
        # Conv2D 
        self.Conv = Conv2D(filters, kernel_size)(inputs)
        # Batch Normalization
        self.BN = BatchNormalization()(self.Conv)
        # Activation Function
        self.AF = Activation("relu")(self.BN)
    def output(self):
        return self.AF

# Depthwise Separable Convolution

In [3]:
class depthwise_separable(Model):
    def __init__(self, filters, kernel_size, inputs):
        super().__init__()
        # Depthwise Convolution 3x3
        self.DWSeparableConv = SeparableConv2D(filters, kernel_size)(inputs)
        # Batch Normalization
        self.BN = BatchNormalization()(self.DWSeparableConv)
        # Activation Function
        self.AF = Activation("relu")(self.BN)

    def output(self):
        return self.AF 

# Separable Squeeze-Expand Block

In [4]:
class SSEModel(Model):
    def __init__(self, squeeze, expand, inputs):
        super().__init__()
        #squeeze
        self.pointwise1 = ConvBNReLU(squeeze, 1, inputs)
        #expand
        self.pointwise2 = ConvBNReLU(expand, 1, self.pointwise1.output())
        self.depthwise = depthwise_separable(expand, 3, self.pointwise1.output())
    def concat(self):
        self.x = Concatenate([self.pointwise2.output(), self.depthwise.output()])
        
        return self.x

# Slim Module

In [5]:
class SlimModule(Model):
    def __init__(self, squeeze, expand, dws, inputs):
        super().__init__()
        self.SSE1 = SSEModel(squeeze, expand, inputs)
        
        self.SkipConnection = Add()([self.SSE1.concat(), inputs])

        self.SSE2 = SSEModel(squeeze, expand)(self.SkipConnection)
        #depthwise separable conv missing?

        self.pool = MaxPool2D(pool_size=(3, 3), strides=(2, 2))(self.SSE2.concat())
        return self.pool

# SlimNet - CNN

In [6]:
#inputShape = (178, 218, 3)

input = Input(shape=(178, 218, 3))

conv1_7x7_s2 = Conv2D(96, kernel_size=(7,7), strides=(2,2))(input)
maaxpool1 = MaxPool2D(pool_size=(3, 3), strides=(2, 2))(conv1_7x7_s2)



SlimNetModel1 = SlimModule(16, 64, 48, maaxpool1)()
SlimNetModel2 = SlimModule(32, 128, 96, SlimNetModel1)
SlimNetModel3 = SlimModule(48, 192, 144, SlimNetModel2)
SlimNetModel4 = SlimModule(64, 256, 192, SlimNetModel3)



ValueError: A merge layer should be called on a list of inputs.